## Features Engineering

### Import Library

In [1]:
import numpy as np
import pandas as pd

### Read Data Training, Data Test

In [2]:
df_train = pd.read_csv("C:\\Users\\Adin\\PYTHON\\klasifikasi\\praproses_data_training.csv")
df_train.head(2)

,itemID,size,color,manufacturerID,price,customerID,salutation,state,returnShipment,age,delivery_time,accountAge,orderYear,orderMonth
0,186,M,blue,25,69.90,794,Mrs,Baden-Wuerttemberg,0,48.0,2.0,981,2012,4
1,71,L,brown,21,69.95,794,Mrs,Baden-Wuerttemberg,1,48.0,2.0,981,2012,4


In [3]:
df = pd.read_csv("C:\\Users\\Adin\\PYTHON\\klasifikasi\\praproses_data_test.csv")
df.head(2)

,itemID,size,color,manufacturerID,price,customerID,salutation,state,age,delivery_time,accountAge,orderYear,orderMonth
0,2347,XL,purple,1,89.9,12489,Mrs,Hesse,50.0,2.0,617,2013,4
1,2741,XL,grey,1,99.9,12489,Mrs,Hesse,50.0,2.0,617,2013,4


## Penambahan atribut

### Data NULL pada atribut age akan diisi oleh rata-rata dari atribut data training, yaitu atribut age dan atribut salutation

In [4]:
df_avg = pd.DataFrame((df_train.groupby('salutation').mean().age))
df_avg

,age
salutation,
Company,49.580357
Family,44.898780
Mr,49.400517
Mrs,49.227598
not reported,50.990000


In [5]:
# DATA TEST
df['age'].fillna(df_train.groupby("salutation")['age'].transform("mean"), inplace = True)
df['age'] = round(df['age']) # membulatkan data

In [6]:
# DATA TRAINING
df_train['age'].fillna(df_train.groupby("salutation")['age'].transform("mean"), inplace = True)
df_train['age'] = round(df_train['age']) # membulatkan data
# df_train.head(5)

### Data NULL pada atribut deliveryTime akan diisi oleh rata-rata lama pengiriman dari data training

In [7]:
mean_delivery_time = round(np.mean(df_train['delivery_time']))
mean_delivery_time

11

In [8]:
# DATA TRAINING
df_train.loc[df_train['delivery_time'].isna(), 'delivery_time'] = mean_delivery_time
df_train.delivery_time.head(6)
# DATA TEST
df.loc[df['delivery_time'].isna(), 'delivery_time'] = mean_delivery_time

### Rasio barang dikembalikan atau tidak berdasarkan atribut CustomerID

In [9]:
resultc = pd.crosstab(df_train.customerID, df_train['returnShipment'])
resultc.columns = ['not_return', 'return']
resultc['total'] = resultc['not_return'] + resultc['return']
resultc['return_customerID'] = resultc['return'] / resultc['total']
resultc['not_return_customerID'] = resultc['not_return'] / resultc['total']
result_mergec = pd.DataFrame()
result_mergec['return_customerID'] = resultc['return_customerID']
result_mergec['not_return_customerID'] = resultc['not_return_customerID']
result_mergec.head(1)

,return_customerID,not_return_customerID
customerID,,
6,0.4,0.6


In [10]:
# DATA TRAINING
df_train = pd.merge(df_train, result_mergec, on = 'customerID', how = 'left')
df_train.head(1)

,itemID,size,color,manufacturerID,price,customerID,salutation,state,returnShipment,age,delivery_time,accountAge,orderYear,orderMonth,return_customerID,not_return_customerID
0,186,M,blue,25,69.9,794,Mrs,Baden-Wuerttemberg,0,48.0,2.0,981,2012,4,0.454545,0.545455


In [11]:
# DATA TEST
df = pd.merge(df, result_mergec, on = 'customerID', how = 'left')
df.head(1)

,itemID,size,color,manufacturerID,price,customerID,salutation,state,age,delivery_time,accountAge,orderYear,orderMonth,return_customerID,not_return_customerID
0,2347,XL,purple,1,89.9,12489,Mrs,Hesse,50.0,2.0,617,2013,4,0.333333,0.666667


### Rasio barang dikembalikan atau tidak berdasarkan atribut deliveryTime

In [12]:
resultdt = pd.crosstab(df_train['delivery_time'], df_train['returnShipment'])
resultdt.columns = ['not_return', 'return']
resultdt['total'] = resultdt['not_return'] + resultdt['return']
resultdt['return_deliveryTime'] = resultdt['return'] / resultdt['total']
resultdt['not_return_deliveryTime'] = resultdt['not_return'] / resultdt['total']
result_mergedt = pd.DataFrame()
result_mergedt['return_deliveryTime'] = resultdt['return_deliveryTime']
result_mergedt['not_return_deliveryTime'] = resultdt['not_return_deliveryTime']
result_mergedt.head(1)

,return_deliveryTime,not_return_deliveryTime
delivery_time,,
0.0,0.513714,0.486286


In [13]:
# DATA TRAINING
df_train = pd.merge(df_train, result_mergedt, on = 'delivery_time', how = 'left')
df_train.head(1)

,itemID,size,color,manufacturerID,price,customerID,salutation,state,returnShipment,age,delivery_time,accountAge,orderYear,orderMonth,return_customerID,not_return_customerID,return_deliveryTime,not_return_deliveryTime
0,186,M,blue,25,69.9,794,Mrs,Baden-Wuerttemberg,0,48.0,2.0,981,2012,4,0.454545,0.545455,0.528936,0.471064


In [14]:
# DATA TEST
df = pd.merge(df, result_mergedt, on = 'delivery_time', how = 'left')
df.head(1)

,itemID,size,color,manufacturerID,price,customerID,salutation,state,age,delivery_time,accountAge,orderYear,orderMonth,return_customerID,not_return_customerID,return_deliveryTime,not_return_deliveryTime
0,2347,XL,purple,1,89.9,12489,Mrs,Hesse,50.0,2.0,617,2013,4,0.333333,0.666667,0.528936,0.471064


### Rasio barang dikembalikan atau tidak berdasarkan atribut itemID

In [15]:
result1 = pd.crosstab(df_train['itemID'], df_train['returnShipment'])
result1.columns = ['not_return', 'return']
result1['total'] = result1['not_return'] + result1['return']
result1['return_itemID'] = result1['return'] / result1['total']
result1['not_return_itemID'] = result1['not_return'] / result1['total']
result_merge1 = pd.DataFrame()
result_merge1['return_itemID'] = result1['return_itemID']
result_merge1['not_return_itemID'] = result1['not_return_itemID']

In [16]:
# DATA TRAINING
df_train = pd.merge(df_train, result_merge1, on = 'itemID', how = 'left')
df_train.head(1)

,itemID,size,color,manufacturerID,price,customerID,salutation,state,returnShipment,age,delivery_time,accountAge,orderYear,orderMonth,return_customerID,not_return_customerID,return_deliveryTime,not_return_deliveryTime,return_itemID,not_return_itemID
0,186,M,blue,25,69.9,794,Mrs,Baden-Wuerttemberg,0,48.0,2.0,981,2012,4,0.454545,0.545455,0.528936,0.471064,0.643087,0.356913


In [17]:
# DATA TEST
df = pd.merge(df, result_merge1, on = 'itemID', how = 'left')
df.head(1)

,itemID,size,color,manufacturerID,price,customerID,salutation,state,age,delivery_time,accountAge,orderYear,orderMonth,return_customerID,not_return_customerID,return_deliveryTime,not_return_deliveryTime,return_itemID,not_return_itemID
0,2347,XL,purple,1,89.9,12489,Mrs,Hesse,50.0,2.0,617,2013,4,0.333333,0.666667,0.528936,0.471064,0.458175,0.541825


### Rasio barang dikembalikan atau tidak berdasarkan atribut ManufacturerID

In [18]:
result = pd.crosstab(df_train['manufacturerID'], df_train['returnShipment'])
result.columns = ['not_return', 'return']
result['total'] = result['not_return'] + result['return']
result['return_manufacturerID'] = result['return'] / result['total']
result['not_return_manufacturerID'] = result['not_return'] / result['total']
result_merge = pd.DataFrame()
result_merge['return_manufacturerID'] = result['return_manufacturerID']
result_merge['not_return_manufacturerID'] = result['not_return_manufacturerID']

In [19]:
# DATA TRAINING
df_train = pd.merge(df_train, result_merge, on = 'manufacturerID', how = 'left')
df_train.head(1)

,itemID,size,color,manufacturerID,price,customerID,salutation,state,returnShipment,age,...,orderYear,orderMonth,return_customerID,not_return_customerID,return_deliveryTime,not_return_deliveryTime,return_itemID,not_return_itemID,return_manufacturerID,not_return_manufacturerID
0,186,M,blue,25,69.9,794,Mrs,Baden-Wuerttemberg,0,48.0,...,2012,4,0.454545,0.545455,0.528936,0.471064,0.643087,0.356913,0.629746,0.370254


In [20]:
# DATA TEST
df = pd.merge(df, result_merge, on = 'manufacturerID', how = 'left')
df.head(1)

,itemID,size,color,manufacturerID,price,customerID,salutation,state,age,delivery_time,...,orderYear,orderMonth,return_customerID,not_return_customerID,return_deliveryTime,not_return_deliveryTime,return_itemID,not_return_itemID,return_manufacturerID,not_return_manufacturerID
0,2347,XL,purple,1,89.9,12489,Mrs,Hesse,50.0,2.0,...,2013,4,0.333333,0.666667,0.528936,0.471064,0.458175,0.541825,0.518587,0.481413


### Rasio barang dikembalikan atau tidak berdasarkan atribut State

In [21]:
results = pd.crosstab(df_train.state, df_train['returnShipment'])
results.columns = ['not_return', 'return']
results['total'] = results['not_return'] + results['return']
results['return_state'] = results['return'] / results['total']
results['not_return_state'] = results['not_return'] / results['total']
result_merges = pd.DataFrame()
result_merges['return_state'] = results['return_state']
result_merges['not_return_state'] = results['not_return_state']
result_merges.head(1)

,return_state,not_return_state
state,,
Baden-Wuerttemberg,0.481873,0.518127


In [22]:
# DATA TRAINING
df_train = pd.merge(df_train, result_merges, on = 'state', how = 'left')
df_train.head(1)

,itemID,size,color,manufacturerID,price,customerID,salutation,state,returnShipment,age,...,return_customerID,not_return_customerID,return_deliveryTime,not_return_deliveryTime,return_itemID,not_return_itemID,return_manufacturerID,not_return_manufacturerID,return_state,not_return_state
0,186,M,blue,25,69.9,794,Mrs,Baden-Wuerttemberg,0,48.0,...,0.454545,0.545455,0.528936,0.471064,0.643087,0.356913,0.629746,0.370254,0.481873,0.518127


In [23]:
# DATA TEST
df = pd.merge(df, result_merges, on = 'state', how = 'left')
df.head(1)

,itemID,size,color,manufacturerID,price,customerID,salutation,state,age,delivery_time,...,return_customerID,not_return_customerID,return_deliveryTime,not_return_deliveryTime,return_itemID,not_return_itemID,return_manufacturerID,not_return_manufacturerID,return_state,not_return_state
0,2347,XL,purple,1,89.9,12489,Mrs,Hesse,50.0,2.0,...,0.333333,0.666667,0.528936,0.471064,0.458175,0.541825,0.518587,0.481413,0.483557,0.516443


### Rata-rata harga berdasarkan artibut itemID

In [24]:
price_avg = pd.DataFrame((df_train.groupby('itemID').mean().price))
price_avg.columns = ["price_itemID"]

In [25]:
# DATA TRAINING
df_train = pd.merge(df_train, price_avg, on = 'itemID', how = 'left')
df_train.head(1)

,itemID,size,color,manufacturerID,price,customerID,salutation,state,returnShipment,age,...,not_return_customerID,return_deliveryTime,not_return_deliveryTime,return_itemID,not_return_itemID,return_manufacturerID,not_return_manufacturerID,return_state,not_return_state,price_itemID
0,186,M,blue,25,69.9,794,Mrs,Baden-Wuerttemberg,0,48.0,...,0.545455,0.528936,0.471064,0.643087,0.356913,0.629746,0.370254,0.481873,0.518127,69.474855


In [26]:
# DATA TEST
df = pd.merge(df, price_avg, on = 'itemID', how = 'left')
df.head(1)

,itemID,size,color,manufacturerID,price,customerID,salutation,state,age,delivery_time,...,not_return_customerID,return_deliveryTime,not_return_deliveryTime,return_itemID,not_return_itemID,return_manufacturerID,not_return_manufacturerID,return_state,not_return_state,price_itemID
0,2347,XL,purple,1,89.9,12489,Mrs,Hesse,50.0,2.0,...,0.666667,0.528936,0.471064,0.458175,0.541825,0.518587,0.481413,0.483557,0.516443,89.9


### Atribut yang datatypenya category diubah menjadi one-hot encoding

In [27]:
df_train = pd.get_dummies(data=df_train, columns=['salutation','size','color','state'])
df = pd.get_dummies(data=df, columns=['salutation','size','color','state'])

### Export

In [28]:
df_train.to_csv('dataset_training.csv', index = False)

In [29]:
df.to_csv('dataset_test.csv', index = False)